In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
rdat = pd.read_csv('RegularSeasonCompactResults.csv')
tdat = pd.read_csv('TourneyCompactResults.csv')
dat = pd.concat([rdat, tdat])

In [3]:
tdat.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [4]:
tdat_history = (tdat.Season.apply(str) + '_' + tdat.loc[:, ['Wteam', 'Lteam']].min(axis=1).apply(str) + '_' + tdat.loc[:, ['Wteam', 'Lteam']].max(axis=1).apply(str))

In [5]:
tdat_history.shape

(2050,)

In [6]:
tdat_results = (tdat.loc[:, ['Wteam', 'Lteam']].min(axis=1) == tdat.Wteam).astype('int')

In [8]:
tdat_results = tdat_results.map({1: 0.99, 0: 0.01})

In [9]:
tdat_results.head()

0    0.99
1    0.99
2    0.99
3    0.99
4    0.99
dtype: float64

In [10]:
tdat_history.name = 'id'
tdat_results.name = 'pred'
gt = pd.concat([tdat_history, tdat_results], axis=1)

In [11]:
submit = pd.read_csv('sample_submission.csv')

In [12]:
submit.head()

,id,pred
0,2013_1103_1107,0.5
1,2013_1103_1112,0.5
2,2013_1103_1125,0.5
3,2013_1103_1129,0.5
4,2013_1103_1137,0.5


In [13]:
gt.head()

,id,pred
0,1985_1116_1234,0.99
1,1985_1120_1345,0.99
2,1985_1207_1250,0.99
3,1985_1229_1425,0.99
4,1985_1242_1325,0.99


In [14]:
submit = pd.merge(submit.loc[:, ['id']], gt, on='id', how='left').fillna(0.5)

In [15]:
submit.pred.value_counts(dropna=False)

0.50    8844
0.01     137
0.99     131
Name: pred, dtype: int64

In [16]:
submit.to_csv('gt.csv', index=False)

In [27]:
dat.loc[(dat.Season == 2013) & ((dat.Wteam == 1103) | (dat.Lteam == 1103))]

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
123897,2013,4,1157,74,1103,70,H,1
124185,2013,10,1329,69,1103,65,N,1
124204,2013,11,1103,82,1421,63,N,0
124326,2013,13,1103,85,1336,60,N,0
124988,2013,27,1103,82,1292,77,H,1
125239,2013,34,1166,77,1103,61,H,0
125323,2013,40,1178,80,1103,73,H,0
125437,2013,43,1103,76,1115,46,H,0
125696,2013,48,1103,87,1156,57,H,0
125720,2013,52,1103,83,1411,68,H,0


In [35]:
history = dat.Season.apply(str) + '_' + dat.loc[:, ['Wteam', 'Lteam']].min(axis=1).apply(str) + '_' + dat.loc[:, ['Wteam', 'Lteam']].max(axis=1).apply(str)

In [36]:
history.shape

(147339,)

In [37]:
history.nunique()

101574

In [17]:
games = dat.loc[:, ['Wteam', 'Lteam']].copy()

In [18]:
games.head()

,Wteam,Lteam
0,1228,1328
1,1106,1354
2,1112,1223
3,1165,1432
4,1192,1447


In [19]:
games.loc[:, 'Min_team'] = games.loc[:, ['Wteam', 'Lteam']].min(axis=1)
games.loc[:, 'Max_team'] = games.loc[:, ['Wteam', 'Lteam']].max(axis=1)

In [20]:
games.head()

,Wteam,Lteam,Min_team,Max_team
0,1228,1328,1228,1328
1,1106,1354,1106,1354
2,1112,1223,1112,1223
3,1165,1432,1165,1432
4,1192,1447,1192,1447


In [21]:
games.loc[:, 'game'] = games.Min_team.apply(lambda x: str(int(x))) + ', ' + games.Max_team.apply(lambda x: str(int(x)))

In [22]:
games.head()

,Wteam,Lteam,Min_team,Max_team,game
0,1228,1328,1228,1328,"1228, 1328"
1,1106,1354,1106,1354,"1106, 1354"
2,1112,1223,1112,1223,"1112, 1223"
3,1165,1432,1165,1432,"1165, 1432"
4,1192,1447,1192,1447,"1192, 1447"


In [23]:
games.loc[:, 'win_lose'] = games.Wteam.apply(lambda x: str(int(x))) + ', ' + games.Lteam.apply(lambda x: str(int(x)))

In [24]:
gpd_win = games.groupby(['game', 'win_lose'])
gpd_game = games.groupby('game')

In [25]:
game_counts = gpd_game.size()
win_lose_history = gpd_win.size()

In [50]:
game_counts.head()

game
0, 112    1
0, 121    1
0, 122    5
0, 129    6
0, 133    1
dtype: int64

In [51]:
win_lose_history.head()

game    win_lose
0, 112  112, 0      1
0, 121  121, 0      1
0, 122  0, 122      3
        122, 0      2
0, 129  129, 0      6
dtype: int64

In [61]:
min_ncontests = 20

win_rate = win_lose_history.div(game_counts.loc[game_counts >= min_ncontests], level=0).dropna()

In [70]:
win_rate.head(20)

game     win_lose
1, 117   1, 117      0.379310
         117, 1      0.620690
1, 206   1, 206      0.253968
         206, 1      0.746032
1, 260   1, 260      0.318182
         260, 1      0.681818
1, 323   1, 323      0.255814
         323, 1      0.744186
1, 327   1, 327      0.207547
         327, 1      0.792453
1, 330   1, 330      0.129032
         330, 1      0.870968
1, 360   1, 360      0.323529
         360, 1      0.676471
1, 39    1, 39       0.181818
         39, 1       0.818182
1, 60    1, 60       0.294118
         60, 1       0.705882
10, 101  10, 101     0.509434
         101, 10     0.490566
dtype: float64

In [71]:
always_win = win_rate.loc[win_rate >= 1]

always_win.name = 'percentage'

always_win = always_win.reset_index()

always_win

In [82]:
always_win.loc[:, 'Wteam_index'] = always_win.win_lose.str.extract(r'(\d+), \d+', expand=False)
always_win.loc[:, 'Lteam_index'] = always_win.win_lose.str.extract(r'\d+, (\d+)', expand=False)

In [83]:
always_win

,game,win_lose,percentage,Wteam_index,Lteam_index
0,"11, 218","11, 218",1.0,11,218
1,"160, 169","160, 169",1.0,160,169
2,"237, 251","237, 251",1.0,237,251
3,"54, 101","54, 101",1.0,54,101
4,"58, 292","292, 58",1.0,292,58
5,"64, 292","292, 64",1.0,292,64
6,"71, 80","80, 71",1.0,80,71


In [91]:
always_win.loc[:, 'Wteam_code'] = always_win.loc[:, 'Wteam_index'].astype('int').map(index2code)
always_win.loc[:, 'Lteam_code'] = always_win.loc[:, 'Lteam_index'].astype('int').map(index2code)

In [92]:
always_win

,game,win_lose,percentage,Wteam_index,Lteam_index,Wteam_code,Lteam_code
0,"11, 218","11, 218",1.0,11,218,1112,1319
1,"160, 169","160, 169",1.0,160,169,1261,1270
2,"237, 251","237, 251",1.0,237,251,1338,1352
3,"54, 101","54, 101",1.0,54,101,1155,1202
4,"58, 292","292, 58",1.0,292,58,1393,1159
5,"64, 292","292, 64",1.0,292,64,1393,1165
6,"71, 80","80, 71",1.0,80,71,1181,1172


In [95]:
teams = pd.read_csv('Teams.csv').set_index('Team_Id')
teams.head()

In [99]:
always_win.loc[:, 'Wteam'] = always_win.Wteam_code.map(teams.Team_Name)
always_win.loc[:, 'Lteam'] = always_win.Lteam_code.map(teams.Team_Name)

In [100]:
always_win

,game,win_lose,percentage,Wteam_index,Lteam_index,Wteam_code,Lteam_code,Wteam,Lteam
0,"11, 218","11, 218",1.0,11,218,1112,1319,Arizona,Northern Arizona
1,"160, 169","160, 169",1.0,160,169,1261,1270,LSU,McNeese St
2,"237, 251","237, 251",1.0,237,251,1338,1352,Pittsburgh,Robert Morris
3,"54, 101","54, 101",1.0,54,101,1155,1202,Clemson,Furman
4,"58, 292","292, 58",1.0,292,58,1393,1159,Syracuse,Colgate
5,"64, 292","292, 64",1.0,292,64,1393,1165,Syracuse,Cornell
6,"71, 80","80, 71",1.0,80,71,1181,1172,Duke,Davidson
